In [5]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [6]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Joseph\tensorflow_datasets\movielens\100k-movies\0.1.1.incompleteJH073B\movielens-train.tfr…

Dataset movielens downloaded and prepared to C:\Users\Joseph\tensorflow_datasets\movielens\100k-movies\0.1.1. Subsequent calls will reuse this data.


In [8]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_id": tf.strings.to_number(x["movie_id"]),
    "user_id": tf.strings.to_number(x["user_id"])
})
movies = movies.map(lambda x: tf.strings.to_number(x["movie_id"]))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
# Build a model.
class Model(tfrs.Model):
    def __init__(self):
        super().__init__()

        # Set up user representation.
        self.user_model = tf.keras.layers.Embedding(
            input_dim=2000, output_dim=64)
        # Set up movie representation.
        self.item_model = tf.keras.layers.Embedding(
            input_dim=2000, output_dim=64)
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.item_model)
            )
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.item_model(features["movie_id"])

        return self.task(user_embeddings, movie_embeddings)

In [14]:
model = Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [15]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
model.fit(train.batch(4096), epochs=5)

# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/5
20/20 [==============================] - 8s 281ms/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0044 - factorized_top_k/top_50_categorical_accuracy: 0.0393 - factorized_top_k/top_100_categorical_accuracy: 0.0877 - loss: 33403.9509 - regularization_loss: 0.0000e+00 - total_loss: 33403.9509
Epoch 2/5
20/20 [==============================] - 6s 290ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0054 - factorized_top_k/top_10_categorical_accuracy: 0.0152 - factorized_top_k/top_50_categorical_accuracy: 0.1041 - factorized_top_k/top_100_categorical_accuracy: 0.2088 - loss: 31087.8544 - regularization_loss: 0.0000e+00 - total_loss: 31087.8544
Epoch 3/5
20/20 [==============================] - 6s 277ms/step - factorized_top_k/top_1_categorical_accuracy: 6.0000e-04 - factorized_top_k/top_5_categorica

{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k/top_10_categorical_accuracy': 0.0020000000949949026,
 'factorized_top_k/top_50_categorical_accuracy': 0.05490000173449516,
 'factorized_top_k/top_100_categorical_accuracy': 0.1517000049352646,
 'loss': 30114.697265625,
 'regularization_loss': 0,
 'total_loss': 30114.697265625}